### Preprocesamiento de los datos

In [3]:
# Importando librerias necesarias
import numpy as np
import pandas as pd
from pandas import ExcelWriter

X = pd.read_excel("/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/Base de datos Lorica.xlsx", 'Hoja5')

#Exploración de la distribución de valores faltantes

missing_values_count = X.isnull().sum()
missing_values_count


Consecutivo                0
N° bolsa                   2
Yacimiento y Ph            0
Prueba de pala            36
RS                       646
Zona                     682
Nivel                      0
Horizonte                682
Profundidad horizonte    682
Elemento                   0
Cocción                    0
T.desgrasante              0
Composicion                0
Densidad                   0
Textura                    0
Grosor (mm)                0
Tamaño                     0
Acabado Interno            0
Acabado Externo            0
Dureza                     0
Color Munsell              0
Color 2                    0
Ahumado                    0
Hollín                     0
Diametro (cm)              0
Forma Vasija               0
Tipo borde                 0
Orientación                0
Forma Labio                0
Técnica                    0
Elemento.1                 0
Localización               0
Baño                       0
Color Baño                 0
Engobe        


#### Calculo del porcentaje de celdas con valores faltantes, sirve para decidir que estrategia utilizar(imputación, eliminación de casos o variables, etc.)


In [5]:

total_cells = np.product(X.shape)
total_cells
total_missing = missing_values_count.sum()
total_missing
porc_mv = total_missing/total_cells*100
porc_mv
total_missing
porc_mv


13.8353581901969

In [17]:
# Eliminación de columnas (variables) con valores faltantes
X1 = X.drop(['N° bolsa' , 'Yacimiento y Ph','Color Munsell', 'Prueba de pala', 'RS', 'Zona', 'Horizonte','Hollín', 'Profundidad horizonte', 'Observaciones', 'Unnamed: 40', 'Unnamed: 41', 'Consecutivo', 'Nivel', 'Color Munsell'], axis=1)

# Construcción de nuevo juego de datos sin valores faltantes
x2 = X1.dropna()
x2 = x2.rename(columns = {'Color 2':'color'})

#comprobación  de que no hay valores faltantes
missing_values_countx2 = x2.isnull().sum()
missing_values_countx2


Elemento             0
Cocción              0
T.desgrasante        0
Composicion          0
Densidad             0
Textura              0
Grosor (mm)          0
Tamaño               0
Acabado Interno      0
Acabado Externo      0
Dureza               0
color                0
Ahumado              0
Diametro (cm)        0
Forma Vasija         0
Tipo borde           0
Orientación          0
Forma Labio          0
Técnica              0
Elemento.1           0
Localización         0
Baño                 0
Color Baño           0
Engobe               0
Color Engobe         0
Estilo Cerámico      0
Cronologia           0
Color Eng o baño2    0
dtype: int64

#### Despues de eliminar filas y columnas más problemáticas, nos queda un dataframe con un shape de 680 casos y 28 variables, sin datos faltantes (se han manejado los valores faltantes manualmente, sin recurrir a ningún tipo de imputación, simplemente eliminandolos)

In [14]:
x2.shape
#guardar archivo en formato excel.
#x2.to_excel('/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/bd_lorica_preproceso1.xlsx', sheet_name='preproceso')
#x2.to_excel('/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/bd_lorica_preproceso.xlsx', sheet_name='preproceso')


(680, 28)

#### Proceso de transformación de variables categóricas en numéricas y normalización de las variables numéricas.

In [22]:
#Libreria usada para el proceso de one-hot encoding
from sklearn.preprocessing import OneHotEncoder
#Seleccionamos solo las variables categóricas. No tiene sentido aplicar el proceso a las numéricas
x2_object = x2.select_dtypes(include=[np.object])
#Además eliminamos las variables "Cronología" y " Estilo cerámico" que seran usadas despues sin codificar
x3_object = x2_object.drop(['Cronologia', 'Estilo Cerámico'], axis=1)
x3_object

#variables dummy (one-hot encoding)

one_hot = pd.get_dummies(x3_object) 


#  restitución de las columnas númericas al datasetaset transformado
one_hot['Grosor (mm)'] = x2['Grosor (mm)']
one_hot['Tamaño'] = x2['Tamaño']
one_hot 

#NORMALIZACIÓN DE LOS DATOS PARA QUE TENGAN LA MISMA UNIDAD DE MEDIDA
from sklearn.preprocessing import MinMaxScaler
x2_number = x2.select_dtypes(include=[np.number])

scaler = MinMaxScaler()
scaler.fit(x2_number)
col_escaladas = scaler.transform(x2_number)
col = pd.DataFrame(col_escaladas)
col.columns = ['grosor_escaladao', 'tamaño_escalado'] 
#incorporación de las variables escaladas a l conjunto de datos.
one_hot['grosor_escalado'] = col['grosor_escaladao']
one_hot['tamaño_escalado'] = col['tamaño_escalado']
#one_hot.to_excel('/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/bd_lorica_preproceso_dummy_estandarizado.xlsx', sheet_name='scaled')
one_hot.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 680 entries, 0 to 681
Columns: 150 entries, Elemento_borde to tamaño_escalado
dtypes: float64(2), int64(2), uint8(146)
memory usage: 123.5 KB


#### Transformación de conjunto original a enteros (integer encoding)

In [30]:
# libreria usada para el proceso
from sklearn.preprocessing import LabelEncoder
# separar las variables numéricas del dataset para que no sean afectadas por el encoding
x3 = x2.drop(['Grosor (mm)', 'Tamaño'], axis=1)

#aplicación del procedimiento
le = LabelEncoder()

enteros = x3.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand') #usamos la función lambda para pasar la función  le.fi_transfomr
# a todas las columnas



#  restituí las columnas númericas al datasetaset
enteros['Grosor (mm)'] = x2['Grosor (mm)']
enteros['Tamaño'] = x2['Tamaño']
enteros.head()

#enteros.to_excel('/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/bd_lorica_preproceso_integerencod.xlsx', sheet_name='labelencod')

,Elemento,Cocción,T.desgrasante,Composicion,Densidad,Textura,Acabado Interno,Acabado Externo,Dureza,color,...,Localización,Baño,Color Baño,Engobe,Color Engobe,Estilo Cerámico,Cronologia,Color Eng o baño2,Grosor (mm),Tamaño
0,1,1,2,6,1,0,2,2,2,0,...,2,3,4,0,2,5,4,2,7,3
1,1,0,2,3,0,0,2,2,2,15,...,2,3,4,0,2,5,4,2,12,5
2,1,0,2,3,0,0,2,2,2,15,...,2,3,4,0,2,5,4,2,8,3
3,0,0,2,3,0,0,1,2,2,15,...,2,3,4,0,2,5,4,2,10,3
4,1,0,2,3,1,0,1,1,0,0,...,2,3,4,0,2,5,4,2,10,3


#### Se ha realizado la transformación del conjunto de datos categóricos en enteros. Ahora repetimos el proceso de normalizaicón de variables numéricas

In [29]:


#scaler = MinMaxScaler()
scaler.fit(x2_number)
col_escaladas = scaler.transform(x2_number)
col = pd.DataFrame(col_escaladas)
col.columns = ['grosor_escaladao', 'tamaño_escalado'] 
enteros['grosor_escaladao'] = col['grosor_escaladao']
enteros['tamaño_escalado'] = col['tamaño_escalado']
#c.to_excel('/media/dsg/TOSHIBA EXT/MASTER/TFM/caso de estudio/bd_lorica_preproceso_integerencod_estandarizado.xlsx', sheet_name='scaled')
enteros.head()

,Elemento,Cocción,T.desgrasante,Composicion,Densidad,Textura,Acabado Interno,Acabado Externo,Dureza,color,...,Color Baño,Engobe,Color Engobe,Estilo Cerámico,Cronologia,Color Eng o baño2,Grosor (mm),Tamaño,grosor_escaladao,tamaño_escalado
0,1,1,2,6,1,0,2,2,2,0,...,4,0,2,5,4,2,7,3,0.333333,0.142857
1,1,0,2,3,0,0,2,2,2,15,...,4,0,2,5,4,2,12,5,0.611111,0.428571
2,1,0,2,3,0,0,2,2,2,15,...,4,0,2,5,4,2,8,3,0.388889,0.142857
3,0,0,2,3,0,0,1,2,2,15,...,4,0,2,5,4,2,10,3,0.500000,0.142857
4,1,0,2,3,1,0,1,1,0,0,...,4,0,2,5,4,2,10,3,0.500000,0.142857
